In [1]:
# устанавливаем библиотеки

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [ ]:
# читаем данные из таблиц

df = pd.read_csv(r'C:\Users\b\OneDrive\Рабочий стол\chat_messages_202506111041.csv')
df_emp = pd.read_csv(r'C:\Users\b\OneDrive\Рабочий стол\managers_202506111043.csv')
df_rop = pd.read_csv(r'C:\Users\b\OneDrive\Рабочий стол\rops_202506111043.csv')

In [3]:
df

,message_id,type,entity_id,created_by,created_at
0,"""01jb7da570sf4f65xdf0ptvv9q""",incoming_chat_message,37556493,0,1730046924
1,"""01jb3t6bc8gvyfnd063nd91ng4""",incoming_chat_message,37549491,0,1729926213
2,"""01jb6gj9ngwk0ybbmg9w90pbqg""",incoming_chat_message,37531455,0,1730016782
3,"""01jb75bkprk0d7hht1g5vy1pm4""",incoming_chat_message,37553371,0,1730038583
4,"""01jb3t5xprbph1c6pym8ervxpm""",incoming_chat_message,37548675,0,1729926199
...,...,...,...,...,...
18924,"""01jb7wekgjash2x5438e4ver43""",outgoing_chat_message,37534717,10262513,1730062798
18925,"""01jb7we9ap70azqhbmqsvxqafh""",outgoing_chat_message,37557233,6780177,1730062787
18926,"""01jb7wdx5x61z0atwjxzff4axn""",outgoing_chat_message,37554115,11448610,1730062775
18927,"""01jb7wd11kcm6500assks44nwk""",outgoing_chat_message,37556525,6780177,1730062746


In [4]:
# избавимся от строк, где 'created_by'=0 и 'type'='outgoing_chat_message', т.к. это определенно ошибка
# затем отсортируем данные по номеру клиента и времени сообщения
# а после создадим поле с ранжированием ответов

df = df[~((df['created_by']==0) & (df['type']=='outgoing_chat_message'))]
df = df.sort_values(['entity_id', 'created_at'], ascending=[True, True])
df['new_rank'] = df.groupby(['entity_id']).cumcount()

In [5]:
df

,message_id,type,entity_id,created_by,created_at,new_rank
3107,"""01jb4pjb1g22ts7z6arcjv11w6""",incoming_chat_message,35810509,0,1729955966,0
4102,"""01jb4pncn7gm1re1zdxkzhnp6s""",outgoing_chat_message,35810509,10262493,1729956065,1
6430,"""01jb4qedfgxnd2yqr4w26hs4q5""",incoming_chat_message,35810509,0,1729956886,2
6639,"""01jb4qj2skc2wjfdm8qaf69yjh""",outgoing_chat_message,35810509,10262493,1729957006,3
17586,"""01jb7gxwb0sepv9vtekrgj3v6d""",incoming_chat_message,35945535,0,1730050716,0
...,...,...,...,...,...,...
18914,"""01jb7w5npw67ghth0ctgqkvrzr""",outgoing_chat_message,37557215,6744792,1730062505,1
18864,"""01jb7vxxz0djjdc4qjezwhycev""",incoming_chat_message,37557225,0,1730062252,0
18908,"""01jb7w3wcr7w96me3k4t5y9a3w""",incoming_chat_message,37557229,0,1730062447,0
18925,"""01jb7we9ap70azqhbmqsvxqafh""",outgoing_chat_message,37557233,6780177,1730062787,0


In [6]:
# избавимся от записей, где беседа начинается с ответа менеджера на неясно какой вопрос
# затем обновим поле с ранжированием сообщений по номеру клиента
# после чего создадим поле, в котором идентифицируем "вопрос-ответ" исходя из условия задачи

df = df[~((df['new_rank']==0) & (df['type']=='outgoing_chat_message'))]
df = df.drop(['new_rank'], axis = 1)
df['new_rank'] = df.groupby(['entity_id']).cumcount()
df['rank'] = df.groupby(['entity_id', 'type']).cumcount()

In [7]:
df

,message_id,type,entity_id,created_by,created_at,new_rank,rank
3107,"""01jb4pjb1g22ts7z6arcjv11w6""",incoming_chat_message,35810509,0,1729955966,0,0
4102,"""01jb4pncn7gm1re1zdxkzhnp6s""",outgoing_chat_message,35810509,10262493,1729956065,1,0
6430,"""01jb4qedfgxnd2yqr4w26hs4q5""",incoming_chat_message,35810509,0,1729956886,2,1
6639,"""01jb4qj2skc2wjfdm8qaf69yjh""",outgoing_chat_message,35810509,10262493,1729957006,3,1
17586,"""01jb7gxwb0sepv9vtekrgj3v6d""",incoming_chat_message,35945535,0,1730050716,0,0
...,...,...,...,...,...,...,...
18839,"""01jb7vgcc06pd71nvyag5ck7ma""",incoming_chat_message,37557215,0,1730061808,0,0
18914,"""01jb7w5npw67ghth0ctgqkvrzr""",outgoing_chat_message,37557215,6744792,1730062505,1,0
18864,"""01jb7vxxz0djjdc4qjezwhycev""",incoming_chat_message,37557225,0,1730062252,0,0
18908,"""01jb7w3wcr7w96me3k4t5y9a3w""",incoming_chat_message,37557229,0,1730062447,0,0


In [8]:
# создадим поле, в котором будет указано время ответа на сообщение. Сделаем это с помощью группировки и смёржим

df2 = df.groupby(['entity_id', 'rank'])['created_at'].min().reset_index()
df = pd.merge(df, df2, how='left', on=['entity_id', 'rank'])

In [9]:
# почистим наш датасет от ненужных строк и столбцов
# также создадим столбцы с корректными датой и временем вопроса и ответа

df = df[~(df['created_by'] == 0)]
df['time_answer'] = df['created_at_x'].apply(lambda x: datetime.fromtimestamp(x))
df['time_question'] = df['created_at_y'].apply(lambda x: datetime.fromtimestamp(x))
df = df.drop(['message_id', 'new_rank', 'rank', 'created_at_x', 'created_at_y'], axis=1)

In [10]:
# напишем функцию, в которой определим условия для расчета времени ответа 
# исходя из условий задачи и времени работы менеджеров

def count_diff (df):
    if (pd.to_datetime(df['time_answer']).day > pd.to_datetime(df['time_question']).day) & (pd.to_datetime(df['time_answer']).hour >=9) & (pd.to_datetime(df['time_answer']).minute >=30):
        return df['time_answer'] - df['time_question'] - timedelta(hours=9, minutes=30)
    elif (pd.to_datetime(df['time_answer']).day > pd.to_datetime(df['time_question']).day) & (pd.to_datetime(df['time_answer']).hour < 9) & (pd.to_datetime(df['time_answer']).minute < 30):
        return datetime(pd.to_datetime(df['time_answer']).year, pd.to_datetime(df['time_answer']).month, pd.to_datetime(df['time_answer']).day,
                        0, 0, 0) - df['time_question']
    else:
        return df['time_answer'] - df['time_question']

In [11]:
# с помощью функции создадим столбец, в котором считается разница во времени между вопросом и ответом

df['diff'] = df.apply(count_diff, axis=1)

In [12]:
# далее найдем среднее время ответа по сотрудникам 
# и дополним данные информацией из других таблиц об их именах и руководителях

data = df.groupby('created_by')['diff'].mean().reset_index()
data = pd.merge(data, df_emp, left_on='created_by', right_on='mop_id').drop('mop_id', axis=1)
data = pd.merge(data, df_rop, how='left', on='rop_id')

In [13]:
# отсортируем данные по имени руководителя и менеджера

data.sort_values(by=['rop_name', 'name_mop'], ascending=[True,True], inplace=True)

In [14]:
# посмотрим на результат

data

,created_by,diff,name_mop,rop_id,rop_name
19,11490270,0 days 00:27:50.865533230,Аня и Ксюша,1,Катя РОП
18,11448610,0 days 00:25:32.146408839,Вика и Катя,1,Катя РОП
0,6645315,0 days 00:17:10.400952380,Гюнель и Илина,1,Катя РОП
10,10262513,0 days 00:32:32.739495798,Ира и Варя,1,Катя РОП
20,11490278,0 days 00:16:45.299270072,Ксюша и Джамиля,1,Катя РОП
11,10465254,0 days 00:06:39.219135802,Настя и Даша,1,Катя РОП
1,6744792,0 days 00:23:19.209411764,Юля и Наташа,1,Катя РОП
13,11188802,0 days 00:59:27.801444043,Даша и Влада,2,Полина РОП
2,6780177,0 days 00:21:01.680000,Даша и Даша,2,Полина РОП
5,7996770,0 days 00:11:42.597402597,Лиза и Ева,2,Полина РОП
